Deriving z.

In [39]:
# modifying "reading in financial data" to 32 assets and right T value

import numpy as np
import pandas as pd

# You may choose to select different parameters/values
number_assets = 32
T  = 392
# Read returns
df = pd.read_csv('returns_data.txt',delim_whitespace=True)

Rraw = df.values.T

# Select the first N,T assets and scenarios, you may choose a different strategy if you would like to do so.
R = Rraw[:number_assets,:T]

# Expected return of each asset
expected_returns = np.mean(R, axis = 1)

# Covariance matrix of asset returns
covariance_matrix = np.cov(R)

/tmp/ipykernel_512/3483040339.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv('returns_data.txt',delim_whitespace=True)


In [40]:
# import random weight function from random_weights.ipynb

def generate_random_weights(number_of_assets):
    weights = np.random.rand(number_of_assets) # Generate random numbers
    weights /= np.sum(weights) # Normalize so that the sum is 1
    return weights

# Test the function with 32 assets
n_assets = 32
random_weights = generate_random_weights(n_assets)
random_weights, sum(random_weights)


(array([0.04530294, 0.03248244, 0.01591696, 0.05819326, 0.05515743,
        0.0018886 , 0.04133333, 0.04105249, 0.00532421, 0.02746209,
        0.06062974, 0.02659319, 0.04392856, 0.00207729, 0.05706425,
        0.00622429, 0.00509702, 0.0474765 , 0.02714145, 0.06078049,
        0.05412684, 0.00992554, 0.00093757, 0.00173598, 0.01410071,
        0.02860928, 0.05080505, 0.00879419, 0.01877773, 0.03859788,
        0.05160082, 0.06086186]),
 1.0)

In [41]:
# Redefining the calculate_historical_VaR function
def calculate_historical_VaR(weights, mu_R, confidence_level=0.95):
    # Calculate portfolio returns for the given weights
    portfolio_returns = np.dot(mu_R, np.transpose(np.array(weights)))
    # Calculate the VaR at the specified confidence level
    VaR = np.percentile(portfolio_returns, 100 * (1 - confidence_level))
    return -VaR

In [42]:
# Define a function to simulate the random generation of weights and calculation of VaR
def simulate_VaR(num_simulations, num_assets, mu_R, confidence_level=0.95):
    VaR_results = []
    weights_list = []

    for _ in range(num_simulations):
        weights = generate_random_weights(num_assets)
        VaR = calculate_historical_VaR(weights, mu_R, confidence_level)
        VaR_results.append(VaR)
        weights_list.append(weights)
    
    return VaR_results, weights_list

In [43]:
# Example usage
num_simulations = 1000  # The number of times we want to run the simulation
num_assets = 32  # Number of assets in the portfolio

# We need the mu_R matrix to run the simulations which should be the historical returns matrix
# For the sake of the example, let's create a dummy mu_R with random values
# Normally, you would use actual historical returns data for your assets here
mu_R = np.random.randn(1000, num_assets)  # 1000 scenarios, 32 assets

# Run the simulation
VaR_results, weights_list = simulate_VaR(num_simulations, num_assets, mu_R)

# Display the first few results to ensure it's working
VaR_results[:5], weights_list[:5]

([0.3201754590963047,
  0.3040817367921548,
  0.33570642951204405,
  0.324621105208764,
  0.35256804169434885],
 [array([0.03519249, 0.03290085, 0.01317292, 0.0536117 , 0.05335179,
         0.0282839 , 0.04913777, 0.03639204, 0.00584395, 0.00473127,
         0.01556526, 0.05040084, 0.00158258, 0.05193638, 0.01152281,
         0.0274181 , 0.04280595, 0.02104865, 0.04392498, 0.01501355,
         0.00104685, 0.03588135, 0.04501165, 0.01426848, 0.03105206,
         0.04519589, 0.02398323, 0.03468755, 0.03416542, 0.04537202,
         0.04764116, 0.04785654]),
  array([0.04150249, 0.04927715, 0.04050631, 0.01704145, 0.02052018,
         0.02598609, 0.03970675, 0.04701215, 0.02573006, 0.03526343,
         0.0118426 , 0.0231728 , 0.03486327, 0.00175815, 0.02021195,
         0.04307619, 0.01233438, 0.04928072, 0.0216664 , 0.01882121,
         0.05057916, 0.02193385, 0.02624362, 0.01480431, 0.04967795,
         0.01678528, 0.04148598, 0.03736406, 0.03326363, 0.03940288,
         0.03435846, 0.05

In [45]:
# let's now actually calculate z

# Parameters
number_of_assets = 32
number_of_portfolios = 1000  # The number of random portfolios you want to generate
confidence_level = 0.95

# Assuming mu_R is the matrix of historical returns for the assets
# Replace this with your actual historical returns data
mu_R = expected_returns  # This should be the historical returns data for your assets

# Generate the random weights for the portfolios and calculate the VaR for each
VaRs = []
for _ in range(number_of_portfolios):
    weights = generate_random_weights(number_of_assets)
    VaR = calculate_historical_VaR(weights, mu_R, confidence_level)
    VaRs.append(VaR)

# Calculate z
mean_VaR = np.mean(VaRs)
median_VaR = np.median(VaRs)
z = mean_VaR  # or median_VaR, depending on your specific requirements

print(f"The mean VaR at a {confidence_level*100}% confidence level is: {mean_VaR}")
print(f"The median VaR at a {confidence_level*100}% confidence level is: {median_VaR}")


The mean VaR at a 95.0% confidence level is: -0.0010112892735360864
The median VaR at a 95.0% confidence level is: -0.0010098220472010626
